In [105]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

df = pd.read_csv('../data/merged_data.csv')
reg = LinearRegression()

df['landplot'].fillna(0, inplace=True)
df['facades'].fillna(0, inplace=True)
df['Living area'].fillna(0, inplace=True)

X = df.drop(['price', 'Zip code', 'Kitchen', 'type', 'subtype', 'condition', 'city', 'province',], axis=1).to_numpy()
y = df['price'].to_numpy()
y = y.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

reg.fit(X_train, y_train)
print("Training score:", reg.score(X_train, y_train))
y_prediction = reg.predict(X_test)
print("Testing score:", reg.score(X_test, y_test))

print("Using 4 features and 1 (price)target: 'Bedroom count', 'Living surface area', 'Landplot area', and 'Terrace'.")


Training score: 0.38281404477072045
Testing score: 0.3750233021765502
Using 4 features and 1 (price)target: 'Bedroom count', 'Living surface area', 'Landplot area', and 'Terrace'.
